In [1]:
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin

# CatBoost
from catboost import CatBoostRegressor

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

# Utility
import os
import time
import datetime
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

import warnings
warnings.filterwarnings('ignore')

### target
 ECLO(Equivalent Casualty Loss Only) : 인명피해 심각도

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1
본 대회에서는 사고의 위험도를 인명피해 심각도로 측정

### 사고유형
- 충돌 : 다른 방향에서 달리는 차끼리 부딪힘
- 추돌 : 같은 방향으로 달리는 차끼리 부딪힘
- 전복 : 180도 뒤집어짐 >> 차의 윗면이 바닥에 닿음
- 전도 : 90도 뒤집어짐 >> 차의 측면이 바닥에 닿음

### 도로형태
- '단일로 
    - '기타' , 터널' , 지하차도(도로)내 , 교량 , 고가도로위
- '교차로 
    - 교차로안', 교차로횡단보도내 , 교차로부근
- '기타 
    - 기타', 
- '주차장 - 주차장
- '미분류 - 미분류'

### 주차장 급지구분
- 1,2,3 급지가 있음. -> merge한 거는 시군구별 해당 급지의 주차장 개수 합임
- 1급지 = 철도 환승역 300m, 단일역 100m 이내
- 2급지 = " 500m, " 300m 이내
- 3급지 = 그외
> 1급지일수록 좋은 주차장

### Insights
1. 공휴일 및 주말에 높다
2.  사고 시간대(20~03시)에높다
3.  연도에 따른 평균값은 낮아지는 추세이다
4.  가해자가 젊을수록 과속을 많이한다+ 속도에 대한 법규위반일수록 eclo가 높다
5.  특정 동에서 사고가 많이 발생하는데 도로형태(고속도로?)와 관련되어 있을 수 있다
    - 단일로 중에서 (터널, 고가도로) 처럼 '과속' 가능성이 높은 도로형태에서 평균 ECLO가 최상위권, 나머지 단일로는 평균이 낮음
    - 교차로는 전체 평균 eclo가 5점대로 중상위권.  
6.  나머지는 eda ㅋㄷ공유 참고


In [21]:
path = '../data/daegu/'
train_org = pd.read_csv(path+'train.csv')
test_org = pd.read_csv(path+'test.csv')
submit = pd.read_csv(path+'sample_submission.csv')

In [22]:
# Baseline code 2
### train, test의 시간정보와 공간정보를 분류하고
### 공간정보를 기준으로 위에서 만든 데이터셋과 merge

train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

# 도로형태 나누지 말자.
# road_pattern = r'(.+) - (.+)'

# train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
# train_df = train_df.drop(columns=['도로형태'])

# test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
# test_df = test_df.drop(columns=['도로형태'])

In [23]:
# Baseline code 1
### 보안등, 어린이 보호구역, 주차장 정보를 주소 기준으로 분류
light_df = pd.read_csv(path+'external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
light_df[['도시', '구', '동', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])
light_df = light_df.groupby(['도시', '구', '동']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)
light_df = light_df.rename({'설치개수':'보안등개수'})
train_df = pd.merge(train_df, light_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, light_df, how='left', on=['도시', '구', '동'])

In [25]:
child_area_df = pd.read_csv(path+'external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
child_area_df[['도시', '구', '동', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])
child_area_df = child_area_df.groupby(['도시', '구', '동']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)
train_df = pd.merge(train_df, child_area_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['도시', '구', '동'])

In [27]:
### ??? 주차장 정보가 필요할까? 
# 가설 : 주차장이 많을수록 도로변 "불법주정차" 수가 줄어 
# 시야 확보에 도움이 될 수 있다. -> ECLO가 낮을 것
# 📢가설 기각... 의미 없음.
parking_df = pd.read_csv(path+'external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])
location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
parking_df[['도시', '구', '동', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])
parking_df = parking_df.groupby(['도시', '구', '동']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)
train_df = pd.merge(train_df, parking_df, how='left', on=['도시', '구', '동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['도시', '구', '동'])

In [26]:
train_df.columns, train_org.columns

(Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',
        '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',
        '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',
        'ECLO', '연', '월', '일', '시간', '도시', '구', '동', '설치개수', '급지구분_1', '급지구분_2',
        '급지구분_3', 'cnt'],
       dtype='object'),
 Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
        '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
        '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
        '경상자수', '부상자수', 'ECLO'],
       dtype='object'))

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           39609 non-null  object 
 1   요일           39609 non-null  object 
 2   기상상태         39609 non-null  object 
 3   노면상태         39609 non-null  object 
 4   사고유형         39609 non-null  object 
 5   사고유형 - 세부분류  39609 non-null  object 
 6   법규위반         39609 non-null  object 
 7   가해운전자 차종     39609 non-null  object 
 8   가해운전자 성별     39609 non-null  object 
 9   가해운전자 연령     39609 non-null  object 
 10  가해운전자 상해정도   39609 non-null  object 
 11  피해운전자 차종     38618 non-null  object 
 12  피해운전자 성별     38618 non-null  object 
 13  피해운전자 연령     38618 non-null  object 
 14  피해운전자 상해정도   38618 non-null  object 
 15  사망자수         39609 non-null  int64  
 16  중상자수         39609 non-null  int64  
 17  경상자수         39609 non-null  int64  
 18  부상자수         39609 non-null  int64  
 19  ECLO